## 2. Data Processing

#### A360 MDK interface

In [1]:
a360ai

<A360 AI Interface for project: Customer Churn>

In [2]:
# Get default data repo
DATAREPO_LIST = a360ai.list_datarepos()
DATAREPO = DATAREPO_LIST['name'][0]
DATAREPO

'Customer Churn'

In [3]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler

In [4]:
import warnings
warnings.filterwarnings('ignore')

### 1. Set Default Data Repo and load data

In [5]:
a360ai.set_default_datarepo(DATAREPO)

In [6]:
a360ai.list_datasets()

,base_name,extension,size
1,Reward-Customer-Churn.csv,csv,736215.0
2,X.parquet,parquet,174436.0
3,X_drift.parquet,parquet,25692.0
4,y.parquet,parquet,42876.0


In [7]:
df = a360ai.load_dataset("Reward-Customer-Churn.csv")

In [8]:
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,MobileOnRecord,AddressOnRecord,LinkedAccount,TwoFactorAuth,...,DeviceProtection,TechSupport,NewsletterSubscribe,PaperlessBilling,Contract,LastLogInOneMonth,Generation,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No,Target,No,...,No,No,No,No,Month-to-month,Yes,Gen Z,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,Target,Yes,...,Yes,No,No,No,One year,No,Gen X,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,Target,Yes,...,No,No,No,No,Month-to-month,Yes,Gen X,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No,Target,Yes,...,Yes,Yes,No,No,One year,No,Millennials,42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Starbucks,No,...,No,No,No,No,Month-to-month,Yes,Gen Z,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,Target,Yes,...,Yes,Yes,Yes,Yes,One year,Yes,Gen X,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Starbucks,No,...,Yes,No,Yes,Yes,One year,Yes,Boomers,103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No,Target,Yes,...,No,No,No,No,Month-to-month,Yes,Gen Z,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Starbucks,No,...,No,No,No,No,Month-to-month,Yes,Gen X,74.40,306.6,Yes


### 2. Data cleaning, data engineering, data preprocessing

In [9]:
#Dropping the customerID as it has no use
df.drop('customerID',axis='columns',inplace=True)

In [10]:
df.dtypes

gender                  object
SeniorCitizen            int64
Partner                 object
Dependents              object
tenure                   int64
MobileOnRecord          object
AddressOnRecord         object
LinkedAccount           object
TwoFactorAuth           object
OnlineBackup            object
DeviceProtection        object
TechSupport             object
NewsletterSubscribe     object
PaperlessBilling        object
Contract                object
LastLogInOneMonth       object
Generation              object
MonthlyCharges         float64
TotalCharges            object
Churn                   object
dtype: object

In [11]:
df.TotalCharges.values

array(['29.85', '1889.5', '108.15', ..., '346.45', '306.6', '6844.5'],
      dtype=object)

In [12]:
# On viewing the datatypes, we see that TotalCharges is object, so we change it to integer
df[pd.to_numeric(df.TotalCharges,errors='coerce').isnull()].shape

(11, 20)

In [13]:
#Remove rows with space in TotalCharges
df1 = df[df.TotalCharges!=" "]
df1.shape

(7032, 20)

In [14]:
df1.TotalCharges = pd.to_numeric(df1.TotalCharges)
df1.TotalCharges.dtypes

dtype('float64')

In [15]:
# check the different values in the different columns
for column in df1:
  if df1[column].dtypes == object:  
    print(column+" :",df1[column].unique())

gender : ['Female' 'Male']
Partner : ['Yes' 'No']
Dependents : ['No' 'Yes']
MobileOnRecord : ['No' 'Yes']
AddressOnRecord : ['No' 'Yes']
LinkedAccount : ['Target' 'Starbucks' 'Amazon']
TwoFactorAuth : ['No' 'Yes']
OnlineBackup : ['Yes' 'No']
DeviceProtection : ['No' 'Yes']
TechSupport : ['No' 'Yes']
NewsletterSubscribe : ['No' 'Yes']
PaperlessBilling : ['No' 'Yes']
Contract : ['Month-to-month' 'One year' 'Two year']
LastLogInOneMonth : ['Yes' 'No']
Generation : ['Gen Z' 'Gen X' 'Millennials' 'Boomers']
Churn : ['No' 'Yes']


In [16]:
df1.replace('No internet service','No',inplace=True)
df1.replace('No phone service','No',inplace=True)

In [17]:
# Converting all 'Yes' to 1 and all 'No' to 0
yes_no_columns = ['Partner','Dependents','MobileOnRecord','AddressOnRecord','TwoFactorAuth','OnlineBackup','DeviceProtection',
                  'TechSupport', 'NewsletterSubscribe', 'PaperlessBilling','LastLogInOneMonth','Churn']
            
for col in yes_no_columns:
  df1[col].replace({'Yes':1,'No':0},inplace=True)

In [18]:
df1['gender'].replace({'Female':1,'Male':0},inplace=True)

In [19]:
# One hot encoding for categorical columns
df2 = pd.get_dummies(data=df1,columns=['LinkedAccount','Contract','Generation'])

In [20]:
cols = ['tenure','MonthlyCharges','TotalCharges']
scaler = MinMaxScaler()
df2[cols] = scaler.fit_transform(df2[cols])

In [21]:
X = df2.drop('Churn',axis='columns')
y = df2['Churn']

### 3. Write dataaset to Data Repo

In [22]:
a360ai.write_dataset(X,"X", overwrite=True)
a360ai.write_dataset(y,"y", overwrite=True)

True